In [1]:
!pip install transformers==4.50.0 datasets==3.5.0 huggingface_hub==0.29.0 peft accelerate torch scikit-learn -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch

if torch.cuda.is_available():
    print("GPU 사용 가능 (CUDA)")
    print(f"→ 사용 중인 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU 사용 불가 (현재 CPU 모드)")

GPU 사용 가능 (CUDA)
→ 사용 중인 GPU: Tesla T4


In [3]:
from datasets import load_dataset
klue_tc_train = load_dataset('klue', 'ynat', split = 'train')
klue_tc_eval = load_dataset('klue', 'ynat', split = 'validation')
klue_tc_train, klue_tc_eval

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

(Dataset({
     features: ['guid', 'title', 'label', 'url', 'date'],
     num_rows: 45678
 }),
 Dataset({
     features: ['guid', 'title', 'label', 'url', 'date'],
     num_rows: 9107
 }))

In [4]:
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
klue_tc_train, klue_tc_eval

(Dataset({
     features: ['title', 'label'],
     num_rows: 45678
 }),
 Dataset({
     features: ['title', 'label'],
     num_rows: 9107
 }))

In [5]:
klue_tc_train.features['label']
# ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

klue_tc_train.features['label'].int2str(1)
# '경제'

klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
  batch['label_str'] = klue_tc_label.int2str(batch['label'])
  return batch

klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

klue_tc_train[0]

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

{'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'}

In [6]:
split = klue_tc_train.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_dataset = split['train']
valid_dataset = split['test']
test_dataset = klue_tc_eval
train_dataset, valid_dataset, test_dataset

(Dataset({
     features: ['title', 'label', 'label_str'],
     num_rows: 36542
 }),
 Dataset({
     features: ['title', 'label', 'label_str'],
     num_rows: 9136
 }),
 Dataset({
     features: ['title', 'label'],
     num_rows: 9107
 }))

In [7]:
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

def tokenize_function(examples):
    return tokenizer(examples['title'], padding = 'max_length', truncation = True)

model_id = "beomi/kcbert-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "value"]
)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 300,295 || all params: 109,224,206 || trainable%: 0.2749


Map:   0%|          | 0/36542 [00:00<?, ? examples/s]

Map:   0%|          | 0/9136 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    push_to_hub=False,
    report_to="none"
)
def compute_metrics(eval_pred):
    logits, label = eval_pred
    preds = np.argmax(logits, axis = -1)
    acc = accuracy_score(label, preds)
    return {"accuracy": acc}

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
import time
import torch
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
torch.cuda.reset_peak_memory_stats()
start_time = time.time()
trainer.train()
end_time = time.time()
total_time = end_time - start_time
minutes = total_time // 60
seconds = total_time % 60
gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)
eval_results = trainer.evaluate(test_dataset)
print(f"학습 시간: {minutes}분{seconds}초")
print(f"GPU 메모리 사용량: {gpu_memory:.2f} GB")
print("평가 결과:")
for k, v in eval_results.items():
    print(f"  - {k}: {v:.4f}" if isinstance(v, float) else f"  - {k}: {v}")

<ipython-input-9-ca2cb01184e9>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.600200,0.593230,0.796300


학습 시간: 27.0분38.74787163734436초
GPU 메모리 사용량: 1.60 GB
평가 결과:
  - eval_loss: 0.7816
  - eval_accuracy: 0.7162
  - eval_runtime: 183.1693
  - eval_samples_per_second: 49.7190
  - eval_steps_per_second: 6.2180
  - epoch: 1.0000


In [10]:
import os

def get_dir_size(path):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.isfile(fp):
                total += os.path.getsize(fp)
    return total

# 모델 저장 디렉토리 (Trainer의 output_dir과 같아야 함)
model_dir = "./results"

# 바이트 → 메가바이트 또는 기가바이트
model_size_bytes = get_dir_size(model_dir)
model_size_mb = model_size_bytes / (1024 ** 2)
model_size_gb = model_size_bytes / (1024 ** 3)

print(f"💾 모델 저장 용량: {model_size_mb:.2f} MB ({model_size_gb:.2f} GB)")

💾 모델 저장 용량: 44.38 MB (0.04 GB)
